In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pydataset import data
import statistics
import seaborn as sns
import env
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import scipy
import acquire
import prepare
from scipy import stats

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.linear_model
import sklearn.preprocessing
import warnings
warnings.filterwarnings("ignore")
# importing my personal wrangle module
import wrangle

### 1. Load the tips dataset.

In [2]:
tips = data('tips')
tips.head(1)

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2


- a. Create a column named tip_percentage. This should be the tip amount divided by the total bill.
- b. Create a column named price_per_person. This should be the total bill divided by the party size.

In [3]:
tips['tip_percentage'] = tips.tip/tips.total_bill
tips['price_per_person'] = tips.total_bill/tips['size']
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,3.446667
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587,7.003333
4,23.68,3.31,Male,No,Sun,Dinner,2,0.139780,11.840000
5,24.59,3.61,Female,No,Sun,Dinner,4,0.146808,6.147500


- c. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?

### Tip percentage is going to be the best predictor, but it's directly calculated using tip, so it is target leakage

- d. Use select k best and recursive feature elimination to select the top 2 features for predicting tip amount. What are they?

In [4]:
# SPLIT....which is throwing errors rn.  Will return.


#Test set is .2 of original dataframe
train, test = train_test_split(tips, test_size = .2, random_state=123) # you can't stratify on a continuous!, stratify= tips.tip)
# The remainder is here divided .7 to train and .3 to validate
train, validate = train_test_split(train, test_size=.3, random_state=123)
train.shape,validate.shape,test.shape

((136, 9), (59, 9), (49, 9))

In [5]:
X_train = train.drop(columns=['tip', 'tip_percentage'])
y_train = train.tip
y_train_percentage = train.tip_percentage

X_validate = validate.drop(columns=['tip', 'tip_percentage'])
y_validate = validate.tip
y_validate_percentage = train.tip_percentage

X_test = test.drop(columns=['tip','tip_percentage'])
y_test = test.tip
y_test_percentage = train.tip_percentage

X_train.shape,y_train.shape,X_validate.shape,y_validate.shape,X_test.shape,y_test.shape,

((136, 7), (136,), (59, 7), (59,), (49, 7), (49,))

In [6]:
X_validate.head()

,total_bill,sex,smoker,day,time,size,price_per_person
163,16.21,Female,No,Sun,Dinner,3,5.403333
197,10.34,Male,Yes,Thur,Lunch,2,5.170000
158,25.00,Female,No,Sun,Dinner,4,6.250000
114,23.95,Male,No,Sun,Dinner,2,11.975000
242,22.67,Male,Yes,Sat,Dinner,2,11.335000


In [7]:
# Scale here....

scaler = sklearn.preprocessing.MinMaxScaler()
# Note that we only call .fit with the training data,
# but we use .transform to apply the scaling to all the data splits.
scaler.fit(X_train[['total_bill', 'size', 'price_per_person']])

X_train_scaled = pd.DataFrame(scaler.transform(X_train[['total_bill', 'size', 'price_per_person']]), columns =  ['total_bill', 'size', 'price_per_person'])
X_validate_scaled = pd.DataFrame(scaler.transform(X_validate[['total_bill', 'size', 'price_per_person']]), columns = ['total_bill', 'size', 'price_per_person'])
X_test_scaled = pd.DataFrame(scaler.transform(X_test[['total_bill', 'size', 'price_per_person']]), columns = ['total_bill', 'size', 'price_per_person'])

# Feature engineering begins here

In [8]:
# K Best model here:

from sklearn.feature_selection import SelectKBest, f_regression

# parameters: f_regression stats test, give me 8 features
f_selector = SelectKBest(f_regression, k=2)

# find the top 8 X's correlated with y
f_selector.fit(X_train_scaled, y_train)

X_reduced = f_selector.transform(X_train_scaled)

print(X_train_scaled.shape)
print(X_reduced.shape)

(136, 3)
(136, 2)


In [9]:
# Shorthand version to arrive at X_reduced from the previous cell
X_reduced2 = SelectKBest(f_regression, k=2).fit_transform(X_train_scaled, y_train)
print(X_reduced2.shape)

(136, 2)


In [10]:
# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature = X_train_scaled.loc[:,feature_mask].columns.tolist()

In [11]:
f_feature

['total_bill', 'size']

In [12]:
print(str(len(f_feature)), 'selected features')
print(f_feature)

2 selected features
['total_bill', 'size']


In [13]:
f_support = f_selector.get_support()

print(f_support) 

[ True  True False]


In [14]:
f_feature = X_train_scaled.loc[:,f_support].columns.tolist()

# you could also get the list this way (among many others)
# f_feature = [X_train_scaled.columns.values[i] for i in range(len(feature_mask)) if feature_mask[i]==True]

print(str(len(f_feature)), 'selected features')
print(f_feature)

2 selected features
['total_bill', 'size']


## Using the K Best model, TOTAL_BILL & SIZE are the two top predictors

In [15]:
# Recursive feature elimination model here:

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# initialize the ML algorithm
lm = LinearRegression()

# create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
rfe = RFE(lm, 2)

# fit the data using RFE
rfe.fit(X_train_scaled,y_train)  

# get the mask of the columns selected
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = X_train_scaled.iloc[:,feature_mask].columns.tolist()
rfe_feature

['total_bill', 'price_per_person']

In [16]:
# view list of columns and their ranking

# get the ranks
var_ranks = rfe.ranking_
# get the variable names
var_names = X_train_scaled.columns.tolist()
# combine ranks and names into a df for clean viewing
rfe_ranks_df = pd.DataFrame({'Var': var_names, 'Rank': var_ranks})
# sort the df by rank
rfe_ranks_df.sort_values('Rank')

,Var,Rank
0,total_bill,1
2,price_per_person,1
1,size,2


In [17]:
print(str(len(rfe_feature)), 'selected features')
print(rfe_feature)

2 selected features
['total_bill', 'price_per_person']


## Using RFE, the best feature are actually TOTAL_BILL & PRICE_PER_PERSON

- e. Use select k best and recursive feature elimination to select the top 2 features for predicting tip percentage. What are they?

In [18]:
# K Best model here:

from sklearn.feature_selection import SelectKBest, f_regression

# parameters: f_regression stats test, give me 8 features
f_selector = SelectKBest(f_regression, k=2)

# find the top 8 X's correlated with y
f_selector.fit(X_train_scaled, y_train_percentage)

X_reduced = f_selector.transform(X_train_scaled)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature = X_train_scaled.loc[:,feature_mask].columns.tolist()

f_feature

['total_bill', 'price_per_person']

## Using the tip percentage as y, K Best gives TOTAL_BILL & PRICE_PER_PERSON


In [19]:
# Recursive feature elimination model here:

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# initialize the ML algorithm
lm = LinearRegression()

# create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
rfe = RFE(lm, 2)

# fit the data using RFE
rfe.fit(X_train_scaled,y_train_percentage)  

# get the mask of the columns selected
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = X_train_scaled.iloc[:,feature_mask].columns.tolist()
rfe_feature

['size', 'price_per_person']

## Using the tip percentage as y, RFE gives SIZE & PRICE_PER_PERSON


- f. Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?


- kbest is statistical in nature
- rfe is model based

Not sure actually...assuming recursion is better bc it sounds cooler.

In [36]:
# Ryan's code to spit out the list of features:

def show_features_rankings(X, rfe):
    """
    Takes in a dataframe and a fit RFE object in order to output the rank of all features
    """
    # rfe here is reference rfe from cell 15
    var_ranks = rfe.ranking_
    var_names = X.columns.tolist()
    ranks = pd.DataFrame({'Var': var_names, 'Rank': var_ranks})
    ranks = ranks.sort_values(by="Rank", ascending=True)
    return ranks

In [37]:
show_features_rankings(X_train_scaled, rfe)

,Var,Rank
1,size,1
2,price_per_person,1
0,total_bill,2


### 2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [22]:
def select_kbest(X,y,k):
    '''
    Function that returns top k features of a model using k best model,
    accepting X columns, y columns, and k number of top features.
    
    '''
    # K Best model here:

    from sklearn.feature_selection import SelectKBest, f_regression

    # parameters: f_regression stats test, give me 8 features
    f_selector = SelectKBest(f_regression, k=k)

    # find the top 8 X's correlated with y
    f_selector.fit(X, y)

    X_reduced = f_selector.transform(X)

    # boolean mask of whether the column was selected or not. 
    feature_mask = f_selector.get_support()

    # get list of top K features. 
    f_feature = X.loc[:,feature_mask].columns.tolist()

    return f_feature

In [23]:
select_kbest(X_train_scaled,y_train, 2)


['total_bill', 'size']

### 3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [24]:
def select_rfe(X,y,k):
    '''
    Function that returns top k features of a model using rfe model,
    accepting X columns, y columns, and k number of top features.
    
    '''
    from sklearn.linear_model import LinearRegression
    from sklearn.feature_selection import RFE

    # initialize the ML algorithm
    lm = LinearRegression()

    # create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
    rfe = RFE(lm, k)

    # fit the data using RFE
    rfe.fit(X,y)  

    # get the mask of the columns selected
    feature_mask = rfe.support_

    # get list of the column names. 
    rfe_feature = X.iloc[:,feature_mask].columns.tolist()
    return rfe_feature

In [25]:
select_rfe(X_train_scaled, y_train, 2)

['total_bill', 'price_per_person']

### 4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [26]:
swiss = data('swiss')
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [27]:
# SPLIT....which is throwing errors rn.  Will return.


#Test set is .2 of original dataframe
swiss_train, swiss_test = train_test_split(swiss, test_size = .2, random_state=123) # you can't stratify on a continuous!, stratify= tips.tip)
# The remainder is here divided .7 to train and .3 to validate
swiss_train, swiss_validate = train_test_split(swiss_train, test_size=.3, random_state=123)
train.shape,validate.shape,test.shape

((136, 9), (59, 9), (49, 9))

In [28]:
swiss_X_train = swiss_train.drop(columns=['Fertility'])
swiss_y_train = swiss_train.Fertility

swiss_X_validate = swiss_validate.drop(columns=['Fertility'])
swiss_y_validate = swiss_validate.Fertility

swiss_X_test = swiss_test.drop(columns=['Fertility'])
swiss_y_test = swiss_test.Fertility

swiss_X_train.shape,swiss_y_train.shape,swiss_X_validate.shape,swiss_y_validate.shape,swiss_X_test.shape,swiss_y_test.shape,

((25, 5), (25,), (12, 5), (12,), (10, 5), (10,))

In [29]:
swiss.columns

Index(['Fertility', 'Agriculture', 'Examination', 'Education', 'Catholic',
       'Infant.Mortality'],
      dtype='object')

In [30]:
# Scale here....

scaler = sklearn.preprocessing.MinMaxScaler()
# Note that we only call .fit with the training data,
# but we use .transform to apply the scaling to all the data splits.
scaler.fit(swiss_X_train)

swiss_X_train_scaled = pd.DataFrame(scaler.transform(swiss_X_train), columns = ['Agriculture', 'Examination', 'Education', 'Catholic', 'Infant.Mortality'])
swiss_X_validate_scaled = pd.DataFrame(scaler.transform(swiss_X_validate), columns = ['Agriculture', 'Examination', 'Education', 'Catholic', 'Infant.Mortality'])
swiss_X_test_scaled = pd.DataFrame(scaler.transform(swiss_X_test), columns = ['Agriculture', 'Examination', 'Education', 'Catholic', 'Infant.Mortality'])

In [31]:
select_kbest(swiss_X_train_scaled, swiss_y_train,3)

['Examination', 'Catholic', 'Infant.Mortality']

In [32]:
select_rfe(swiss_X_train_scaled, swiss_y_train,3)

['Agriculture', 'Examination', 'Infant.Mortality']

wow. just wow. 

In [33]:
swiss.index

Index(['Courtelary', 'Delemont', 'Franches-Mnt', 'Moutier', 'Neuveville',
       'Porrentruy', 'Broye', 'Glane', 'Gruyere', 'Sarine', 'Veveyse', 'Aigle',
       'Aubonne', 'Avenches', 'Cossonay', 'Echallens', 'Grandson', 'Lausanne',
       'La Vallee', 'Lavaux', 'Morges', 'Moudon', 'Nyone', 'Orbe', 'Oron',
       'Payerne', 'Paysd'enhaut', 'Rolle', 'Vevey', 'Yverdon', 'Conthey',
       'Entremont', 'Herens', 'Martigwy', 'Monthey', 'St Maurice', 'Sierre',
       'Sion', 'Boudry', 'La Chauxdfnd', 'Le Locle', 'Neuchatel', 'Val de Ruz',
       'ValdeTravers', 'V. De Geneve', 'Rive Droite', 'Rive Gauche'],
      dtype='object')